In [1]:
from ketos.neural_networks.densenet import DenseNetInterface
from ketos.data_handling.data_feeding import BatchGenerator, JointBatchGen
import ketos.data_handling.database_interface as dbi
import tensorflow as tf
import random

2022-01-19 22:20:21.855927: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-19 22:21:11.231694: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-19 22:21:11.237901: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-19 22:21:11.326514: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-01-19 22:21:11.327711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:05:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2

In [2]:
# Check if GPU is detected properly
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [3]:
db_save_path='../../saved_database/'
db_save_filename='test_ds_multiclass_original.h5'

# Open a connection to the database.
db = dbi.open_file(db_save_path+db_save_filename, 'r')
print(db)

../../saved_database/test_ds_multiclass_original.h5 (File) ''
Last modif.: 'Sat Jan 15 17:41:08 2022'
Object Tree: 
/ (RootGroup) ''
/dolphin (Group) ''
/dolphin/data (Table(1390,), fletcher32, shuffle, zlib(1)) ''
/hb (Group) ''
/hb/data (Table(24621,), fletcher32, shuffle, zlib(1)) ''
/kw (Group) ''
/kw/data (Table(49341,), fletcher32, shuffle, zlib(1)) ''
/other (Group) ''
/other/data (Table(41484,), fletcher32, shuffle, zlib(1)) ''



In [4]:
kw_table = dbi.open_table(db, '/kw/data')
hb_table = dbi.open_table(db, '/hb/data')
dolphin_table = dbi.open_table(db, '/dolphin/data')
other_table = dbi.open_table(db, '/other/data')

In [5]:
def transform_batch(X, Y):
  x = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
  y = tf.one_hot(Y, depth=4, axis=1).numpy()
  return x, y

In [6]:
train_ratio = 0.8
# Choose random training and validation index randomly
kw_table_nrows_range = range(kw_table.nrows)
kw_train_indices = random.sample(kw_table_nrows_range, int(kw_table.nrows*train_ratio))
kw_val_indices = [x for x in kw_table_nrows_range if x not in kw_train_indices]

hb_table_nrows_range = range(hb_table.nrows)
hb_train_indices = random.sample(hb_table_nrows_range, int(hb_table.nrows*train_ratio))
hb_val_indices = [x for x in hb_table_nrows_range if x not in hb_train_indices]

dolphin_table_nrows_range = range(dolphin_table.nrows)
dolphin_train_indices = random.sample(dolphin_table_nrows_range, int(dolphin_table.nrows*train_ratio))
dolphin_val_indices = [x for x in dolphin_table_nrows_range if x not in dolphin_train_indices]

other_table_nrows_range = range(other_table.nrows)
other_train_indices = random.sample(other_table_nrows_range, int(other_table.nrows*train_ratio))
other_val_indices = [x for x in other_table_nrows_range if x not in other_train_indices]

In [7]:
batch_size = 32
epochs = 50

# Get the training generators for each data table
kw_generator_train = BatchGenerator(batch_size=batch_size, data_table=kw_table,
                                    output_transform_func=transform_batch,
                                    select_indices=kw_train_indices,
                                    shuffle=True, refresh_on_epoch_end=True)

hb_generator_train = BatchGenerator(batch_size=batch_size, data_table=hb_table,
                                    output_transform_func=transform_batch,
                                    select_indices=hb_train_indices,
                                    shuffle=True, refresh_on_epoch_end=True)

dolphin_generator_train = BatchGenerator(batch_size=batch_size, data_table=dolphin_table,
                                         output_transform_func=transform_batch,
                                         select_indices=dolphin_train_indices,
                                         shuffle=True, refresh_on_epoch_end=True)

other_generator_train = BatchGenerator(batch_size=batch_size, data_table=other_table,
                                       output_transform_func=transform_batch,
                                       select_indices=other_train_indices,
                                       shuffle=True, refresh_on_epoch_end=True)

# Combine all the training generators
# === For training 4-class model ===
# train_generator = JointBatchGen([kw_generator_train, hb_generator_train, dolphin_generator_train, other_generator_train])

# === For training 3-class model ===
train_generator = JointBatchGen([kw_generator_train, hb_generator_train, other_generator_train]) 


In [8]:
# Get the training generators for each data table
kw_generator_val = BatchGenerator(batch_size=batch_size, data_table=kw_table, 
                                    output_transform_func=transform_batch,
                                    select_indices = kw_val_indices,
                                    shuffle=True, refresh_on_epoch_end=True)

hb_generator_val = BatchGenerator(batch_size=batch_size, data_table=hb_table, 
                                    output_transform_func=transform_batch,
                                    select_indices = hb_val_indices,
                                    shuffle=True, refresh_on_epoch_end=True)

dolphin_generator_val = BatchGenerator(batch_size=batch_size, data_table=dolphin_table, 
                                        output_transform_func=transform_batch,
                                        select_indices = dolphin_val_indices,
                                        shuffle=True, refresh_on_epoch_end=True)

other_generator_val = BatchGenerator(batch_size=batch_size, data_table=other_table, 
                                        output_transform_func=transform_batch,
                                        select_indices = other_val_indices,
                                        shuffle=True, refresh_on_epoch_end=True)

# Combine all the training generators
# === For training 4-class model ===
# val_generator = JointBatchGen([kw_generator_val, hb_generator_val, dolphin_generator_val, other_generator_val])

# === For training 3-class model ===
val_generator = JointBatchGen([kw_generator_val, hb_generator_val, other_generator_val])

In [ ]:
#Define the parameters for the early-stopping monitor
monitor = {"metric":"val_loss",                    
          "decreasing": True,
          "period":10,
          "min_epochs":epochs//3,
          "delta" : 0.01,
          "baseline":0.0 }

# Add any version number or name here to save it along with the model name
detector_version='kw_multi_detector_v03'
densenet = DenseNetInterface.build_from_recipe_file("densenet_recipe_1.json")
densenet.checkpoint_dir = "checkpoints_"+detector_version
densenet.log_dir = "log_"+detector_version
densenet.train_generator = train_generator
densenet.val_generator = val_generator
# densenet.early_stopping_monitor = monitor

densenet.train_loop(n_epochs=epochs, log_csv=True, validate=True, early_stopping=False)

2022-01-19 22:23:30.346582: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-19 22:23:30.443052: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200090000 Hz
2022-01-19 22:23:32.554434: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-19 22:23:36.083704: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-19 22:23:36.880924: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-01-19 22:23:50.653762: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-01-19 22:23:50.754910: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform 


Epoch: 1 
train_loss: 0.6052400469779968
train_CategoricalAccuracy: 0.395 train_Precision: 0.000 train_Recall: 0.000 
val_loss: 0.7239161133766174
val_CategoricalAccuracy: 0.278 val_Precision: 0.000 val_Recall: 0.000 


Epoch: 2 
train_loss: 0.5583250522613525
train_CategoricalAccuracy: 0.441 train_Precision: 0.000 train_Recall: 0.000 
val_loss: 0.7251323461532593
val_CategoricalAccuracy: 0.276 val_Precision: 0.000 val_Recall: 0.000 


Epoch: 3 
train_loss: 0.5559195280075073
train_CategoricalAccuracy: 0.443 train_Precision: 0.000 train_Recall: 0.000 
val_loss: 0.718410074710846
val_CategoricalAccuracy: 0.283 val_Precision: 0.000 val_Recall: 0.000 


Epoch: 4 
train_loss: 0.5499709844589233
train_CategoricalAccuracy: 0.449 train_Precision: 0.000 train_Recall: 0.000 
val_loss: 0.7208212018013
val_CategoricalAccuracy: 0.280 val_Precision: 0.000 val_Recall: 0.000 


Epoch: 5 
train_loss: 0.5517396330833435
train_CategoricalAccuracy: 0.447 train_Precision: 0.000 train_Recall: 0.000 
val_l

In [ ]:
densenet.save_model("../../trained_models/"+detector_version+".kt", audio_repr_file="../create_db/spec_config.json")

In [ ]:
db.close()  # close the database connection
